In [67]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from os import walk
import regex as re

In [28]:
def get_filecount(BRAND):
    files = []
    for (dirpath, dirnames, filenames) in walk(f'fabricdotcom/{BRAND}/'):
        files.extend(filenames)
        break
    return len(files)

In [49]:
f = open(f'fabricdotcom/AH/1.html', "r")
data = f.read()
soup = BeautifulSoup(data, 'html.parser')
items = soup.find_all("div", {"class": "product-item"})

In [73]:
item = items[5]

In [91]:
def get_data(BRAND):
    df = pd.DataFrame(columns=['page','number','name','low_stock','not_available','price','url'])
    filecount = get_filecount(BRAND)
    for page in list(np.arange(1,filecount+1)):
        try:
            f = open(f'fabricdotcom/{BRAND}/{page}.html', "r")
            data = f.read()
            soup = BeautifulSoup(data, 'html.parser')
            items = soup.find_all("div", {"class": "product-item"})
            i=0
            for item in items:
                d = {}
                desc = item.find("a", {"class":"brand-text"})
                d['page']=page
                d['number']=i
                d['name'] = desc.string
                price = item.select('span[id*="Price"]')[0].get_text()
                if price:
                    d['price']=price
                else:
                    d['price']=None
                d['url'] = desc['href']
                try:
                    lowstock = item.find("span", {"class": "LowStockLabel"}).find("span")
                    if lowstock:
                        d['low_stock'] = lowstock.string
                except:
                    d['low_stock'] = None
                try:
                    if item.find("span", {"class": "NotAvailableLabel"}):
                        d['not_available'] = True
                    else:
                        d['not_available'] = False
                except:
                    d['not_available'] = False
                df = df.append(d, ignore_index=True)
                i+=1
        except:
            print(f'Error reading file fabricdotcom/{BRAND}/{page}.html')
    df = df.sort_values(by=['not_available','low_stock'],ascending=False).reset_index().rename(columns={'index':'bestseller_rank'})
    df.to_csv(f'data/fabricdotcom_{BRAND}')
    return df

In [92]:
get_data('AH')

,bestseller_rank,page,number,name,low_stock,not_available,price,url
0,24,1,24,Alexander Henry Nicole's Prints Rise & Shine Pink,None,True,$11.62,https://www.fabric.com/buy/0758715/alexander-h...
1,34,1,34,Alexander Henry Cat-finity Spiral Cats Natural...,None,True,$11.88,https://www.fabric.com/buy/0561102/alexander-h...
2,36,1,36,Alexander Henry Beauties and Brains Black,None,True,$11.94,https://www.fabric.com/buy/0758700/alexander-h...
3,65,1,65,Alexander Henry Nicole's Prints Ink Work Tea B...,None,True,$11.48,https://www.fabric.com/buy/0758711/alexander-h...
4,72,1,72,Alexander Henry Deep Sea Day Aqua,None,True,$11.62,https://www.fabric.com/buy/0739919/alexander-h...
...,...,...,...,...,...,...,...,...
229,226,4,1,Alexander Henry Garbo B + W,None,False,$12.28,https://www.fabric.com/buy/0739909/alexander-h...
230,227,4,2,Alexander Henry A Ghastlie Bulb Mint/Pink,None,False,$11.62,https://www.fabric.com/buy/0679301/alexander-h...
231,228,4,3,Alexander Henry Garbo Tea/Black,None,False,$11.36,https://www.fabric.com/buy/0739908/alexander-h...
232,229,4,4,Alexander Henry Rooster Heavy Oxford Black,None,False,$18.58,https://www.fabric.com/buy/0695262/alexander-h...


In [93]:
get_data('MM')

Error reading file fabricdotcom/MM/24.html


,bestseller_rank,page,number,name,low_stock,not_available,price,url
0,6,1,6,"Michael Miller Over the Rainbow Rainbow Hair, ...",None,True,$11.23,https://www.fabric.com/buy/0603725/michael-mil...
1,29,1,29,Michael Miller Cool Cats Little Stripe Black,None,True,$8.50,https://www.fabric.com/buy/0402162/michael-mil...
2,73,1,73,Michael Miller Wee Sparkle Metallic Heart Spri...,None,True,$10.79,https://www.fabric.com/buy/0427846/michael-mil...
3,77,2,2,Michael Miller Home For The Holidays Frosty Fr...,None,True,$11.25,https://www.fabric.com/buy/0619957/michael-mil...
4,90,2,15,Michael Miller Minky Kitties Cream,None,True,$17.89,https://www.fabric.com/buy/0620801/michael-mil...
...,...,...,...,...,...,...,...,...
1659,1633,22,58,Michael Miller Minky Fantasy Bloom Fantasy Blo...,None,False,$18.28,https://www.fabric.com/buy/0725018/michael-mil...
1660,1640,22,65,Michael Miller Minky Monster Mash Monster Mash...,None,False,$17.75,https://www.fabric.com/buy/0563060/michael-mil...
1661,1642,22,67,Michael Miller Minky Little Ladies Triangle Sw...,None,False,$9.78,https://www.fabric.com/buy/0680551/michael-mil...
1662,1647,22,72,Michael Miller Minky Bigfoot Boogy Bigfoot Boo...,None,False,$17.23,https://www.fabric.com/buy/0563087/michael-mil...
